# 🚀 The Edge Pattern Hunter: Semantic Compression & Discovery

**Edge AI 환경**에서 대규모 센서 데이터를 효율적으로 처리하기 위한 **의미론적 압축(Semantic Compression)** 및 **교차 모달 패턴 발견(Cross-modal Pattern Discovery)** 기술을 테스트한다

## 📌 구성 스테이지
1. **Stage 1**: 환경 설정 및 최적화 설치
2. **Stage 2**: LLM 및 데이터 시뮬레이터 로직 정의
3. **Stage 3**: 60일 데이터 생성 및 의미론적 압축 (파일 저장)
4. **Stage 4**: 비교 분석 (Baseline vs Proposed) 및 동적 성능 리포트

## Stage 1: 환경 설정

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

!pip install -q transformers accelerate bitsandbytes pandas numpy torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 41.9 MB/s eta 0:00:00


## Stage 2: 최적화 라이브러리 및 모델 정의

In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import time
import gc

def flush():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

class ColabQwenLLM:
    def __init__(self, model_id="Qwen/Qwen3-8B"):
        print(f"Loading {model_id}...")
        flush()
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            torch_dtype=torch.float16,
            device_map="auto",
            low_cpu_mem_usage=True,
            trust_remote_code=True,
            attn_implementation="sdpa"
        )
        self.tokenizer.pad_token = self.tokenizer.eos_token
        print("✅ 모델 로드 완료!")

    def analyze(self, system_prompt, user_prompt, max_tokens=250, temp=0.3):
        flush()
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]
        text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = self.tokenizer([text], return_tensors="pt").to("cuda")

        start_time = time.time()
        with torch.inference_mode():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temp,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
        latency = time.time() - start_time
        response = self.tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True).strip()

        token_count = inputs['input_ids'].shape[1]
        del inputs, outputs
        flush()
        return response, latency, token_count

def generate_data():
    dates = pd.date_range(start="2024-01-01", periods=17280, freq="5min")
    df = pd.DataFrame({'timestamp': dates})
    df['day'] = df['timestamp'].dt.dayofyear
    df['humidity'] = 45 + 25 * np.sin(df['day']/5) + np.random.normal(0, 5, len(df))
    df['vibration'] = 1.0 + 2.0 * (df['humidity'].shift(288*2) > 70).fillna(0).astype(float)
    df['vibration'] += (df['day'] % 14) * 0.05 + np.random.normal(0, 0.1, len(df))
    df.to_csv("sensor_data.csv", index=False)
    return df

## Stage 3: 데이터 생성 및 의미론적 압축
실제 60일치 데이터를 생성하고 일별 요약 로그를 추출합니다.

In [ ]:
df = generate_data()
llm = ColabQwenLLM()
chronicles = []
unique_dates = pd.to_datetime(df['timestamp']).dt.date.unique()
for d in range(60):
    day_date = unique_dates[d]
    day_data = df[pd.to_datetime(df['timestamp']).dt.date == day_date]
    v_mean, h_max = day_data['vibration'].mean(), day_data['humidity'].max()
    sys_prompt = "You are a factory sensor logger. Describe the status in one Korean sentence focusing on numbers."
    user_prompt = f"Date: {day_date}, Avg Vibration: {v_mean:.2f}mm/s, Peak Humidity: {h_max:.1f}%"
    summary, _, _ = llm.analyze(sys_prompt, user_prompt, max_tokens=150)
    chronicles.append(f"Day {d+1} ({day_date}): {summary}")
    if (d+1) % 20 == 0: print(f"Progress: {d+1}/60 days compressed...")
with open("factory_chronicle.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(chronicles))
print(f"\n✅ 60일치 압축 완료!")

Loading Qwen/Qwen3-8B...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

✅ 모델 로드 완료!
Progress: 20/60 days compressed...
Progress: 40/60 days compressed...
Progress: 60/60 days compressed...

✅ 60일치 압축 완료!


## Stage 4: 비교 분석
OOM 해결을 위한 초압축 데이터 포맷을 사용하여 Baseline과 Proposed의 성능을 한 번에 비교합니다.

In [ ]:
import re
print("[Stage 4] Comparative Analysis: Efficiency & Latency")
flush()

# 1. Baseline Analysis (Naive Verbose)
print("Running Baseline Analytics...")
df_raw = pd.read_csv("sensor_data.csv")
unique_dates = pd.to_datetime(df_raw['timestamp']).dt.date.unique()
raw_stats = "\n".join([f"On {unique_dates[i]}, max humidity was {df_raw[pd.to_datetime(df_raw['timestamp']).dt.date == unique_dates[i]]['humidity'].max():.1f}% and vibration was {df_raw[pd.to_datetime(df_raw['timestamp']).dt.date == unique_dates[i]]['vibration'].mean():.2f}mm/s." for i in range(60)])

sys_b = "You are a data analyst. Analyze only the provider data in Korean. Head-First. Answer directly without thinking steps."
user_b = f"60일 수치 데이터를 보고 습도와 진동의 인과관계를 분석해줘:\n{raw_stats}"
baseline_insight, baseline_time, baseline_tokens = llm.analyze(sys_b, user_b, max_tokens=150)

del raw_stats, df_raw
flush()```

# 2. Proposed Analysis (Pattern Hunter)
print("Running Proposed Analytics...")
with open("factory_chronicle.txt", "r", encoding="utf-8") as f:
    raw_lines = [line.strip() for line in f.readlines() if line.strip()]

short_logs = []
for i, l in enumerate(raw_lines[:60]):
    content = l.split('): ')[1] if '): ' in l else l
    short_logs.append(f"D{i+1}: {content[:90]}")

timeline_compact = "\n".join(short_logs)

sys_p = "Pattern Hunter. Find the specific 'Time Lag' (how many days later) between high humidity and vibration spike in Korean. Head-First. Answer directly with no thinking tags."
user_p = f"다음 60일 로그에서 습도와 진동의 인과관계를 찾아줘:\n{timeline_compact}"
proposed_insight, proposed_time, proposed_tokens = llm.analyze(sys_p, user_p, max_tokens=250)

del short_logs, timeline_compact
flush()

print("\n" + "="*70)
print("📊 DYNAMIC PERFORMANCE COMPARISON (60-DAY)")
print("="*70)
print(f"[BASELINE - Verbose]\n- 입력 토큰: {baseline_tokens} tokens\n- 응답 시간: {baseline_time:.2f}초\n- 분석결과: {baseline_insight}")
print("-"*70)
print(f"[PROPOSED - Pattern Hunter]\n- 입력 토큰: {proposed_tokens} tokens ({(baseline_tokens/proposed_tokens):.1f}배 효율)\n- 응답 시간: {proposed_time:.2f}초 ({(baseline_time/proposed_time):.1f}배 속도 향상)\n- 분석결과: {proposed_insight}")
print("="*70)
print(f"📈 분석 데이터 요약: Proposed 방식은 Baseline 대비 {baseline_tokens - proposed_tokens}개의 토큰을 절감하며 {baseline_time - proposed_time:.2f}초의 지연 시간을 단축했습니다.")
print("="*70)
flush()

[Stage 4] Comparative Analysis: Efficiency & Latency
Running Baseline Analytics...
Running Proposed Analytics...

📊 DYNAMIC PERFORMANCE COMPARISON (60-DAY)
[BASELINE - Verbose]
- 입력 토큰: 2043 tokens
- 응답 시간: 18.36초
- 분석결과: <think>
</think>

습도가 높아질수록 진동이 증가하는 경향이 있음을 확인할 수 있습니다. 특히 습도가 70% 이상일 때 진동이 1.10mm/s 이상으로 상승하는 것을 볼 수 있습니다. 하지만 습도가 감소할 때 진동이 감소하는 것은 아닙니다. 예를 들어, 2024-01-14에서 습도가 70.1%로 감소했지만 진동은 1.12mm/s로 감소하지 않았습니다. 이는 습도와 진동 사이에 직접적인 인과관계가 아니라,
----------------------------------------------------------------------
[PROPOSED - Pattern Hunter]
- 입력 토큰: 1314 tokens (1.6배 효율)
- 응답 시간: 11.76초 (1.6배 속도 향상)
- 분석결과: <think>
</think>

습도가 높은 날 다음으로 진동 피크가 발생하는 날짜 차이를 계산하면, 습도가 최고점에 도달한 D15(2024-01-15) 다음으로 진동 피크가 발생한 D18(2024-01-18)이므로, **3일 후**에 진동 피크가 발생했습니다.
📈 분석 데이터 요약: Proposed 방식은 Baseline 대비 729개의 토큰을 절감하며 6.61초의 지연 시간을 단축했습니다.
